In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# **Mount**

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

In [ ]:
import os
from pathlib import Path

folder = "Colab Notebooks"
project_dir = "PROJECT"

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder / project_dir
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"현재 디렉토리 위치: {os.getcwd()}")

# **SetUp and Download FFHQ model**

In [ ]:
!pip install torch==1.10.0+cu113 torchvision==0.11.0+cu113 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))

!python -V
!nvcc -V

In [ ]:
# git clone

!git clone https://github.com/happy-jihye/Cartoon-StyleGan2.git
os.chdir(f'./Cartoon-StyleGan2')

In [ ]:
import cv2 
import numpy as np
from utils import imshow, tensor2image

!pip install git+https://github.com/n-CLAIR/nsml-local
!pip install ftfy regex tqdm ninja gdown

In [ ]:
from utils import download_pretrained_model

download_pretrained_model(False, "ffhq256.pt")

# fine-tuning 대비하여 backup 만들기
"""
    if not os.path.isdir('networks'):
        os.makedirs('networks')
    from gdown import download as drive_download

    url = "https://drive.google.com/uc?id=1PQutd-JboOCOZqmd95XWxWrO8gGEvRcO"
    networkfile = os.path.join('networks', "ffhq256.pt")
    drive_download(url, networkfile, quiet=False)
"""

# **Generate Random Images using FFHQ model**

In [ ]:
from model import Generator

device='cuda'
main_path = Path(f"{os.getcwd()}")

network1 = 'ffhq256'  
network1 = torch.load(main_path / f"networks/{ network1 }.pt" ) # parameter 가져와서 network1에 저장
g1 = Generator(256, 512, 8, channel_multiplier=2).to(device) # StyleGAN model 만들기
g1.load_state_dict(network1["g_ema"], strict=False) # model에 parameter 전달
trunc1 = g1.mean_latent(4096) # 4096배  randn으로 생성하고 평균값을 저장 => 정규분포의 평균에 가까워지고 이상치 없어짐

outdir = 'results'
if not os.path.isdir(f'{outdir}'):
   os.makedirs(main_path / f'asset/{outdir}', exist_ok=True)

In [ ]:
# 이미지 생성
# shape가 1 * 14 * 512인 이유, get_latent의 의미는 모르겠음..
with torch.no_grad():

    # 임의의 FFHQ 이미지를 생성하는 latent 생성
    latent1 = torch.randn(1, 14, 512, device=device)  # 1 * 14 * 512의 정규분포 latent 생성
    latent1 = g1.get_latent(latent1)  # latent를 g1(FFHQ)에 적용할 수 있도록 변형

    # 입력 latent를 가지고 이미지 생성
    imgs_gen1, _ = g1(
        [latent1],
        input_is_latent=True,
        truncation=0.7,
        truncation_latent=trunc1,
        swap=False,
        swap_layer_num=0,
        randomize_noise=False,
    )

    # 이미지 보여주기
    imshow(tensor2image(imgs_gen1))

# **Fine-tuning FFHQ to Disney_StructureLoss**

In [ ]:
# run.py라고 생각했지만, run.py는 압축된 lmdb를 사용할 경우에 하는 것(아님 말구)
# prepare_data.py로 이미지를 lmdb dataset으로 변경해야 함
# ./IMAGE 내부에 데이터셋 폴더를 삽입 후 실행하면 ./DATA에 dataset 생성

!python prepare_data.py --out ./DATA/KOREAN --size 128,256,512 ./IMAGE/KOREAN_IMAGE

In [ ]:
# colab pro는 다른 gpu를 사용하기 때문에 torch/cuda 버전에서 1.8.0+cu111 수행 불가능(RuntimeError 발생)
# 그렇다고 1.12.0(default)를 사용하면 colab free에서와 같은 오류가 발생
# 그래서 그 중간 어딘가의 버전을 사용해서 어떠한 오류도 발생하지 않도록 함(1.10.0+cu113)
# %pip install torch==1.10.0+cu113 torchvision==0.11.0+cu113 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

!python train.py --batch=2 --ckpt=./networks/ffhq256.pt --structure_loss=2 --freezeD=3 --augment --path=./DATA/KOREAN